# Notebook 4: Symmetric vs. Asymmetric Search
In this notebook we will take a look at the differences between symmetric and asymmetric Search.

You can read up on the differences between symmetric and asymmetric search in our [blogpost](https://www.aleph-alpha.com/luminous-explore-a-model-for-world-class-semantic-representation).

In [19]:
!pip install aleph_alpha_client

from typing import Sequence
from aleph_alpha_client import ImagePrompt, AlephAlphaModel, SemanticEmbeddingRequest, SemanticRepresentation, Prompt
import math
import numpy as np
import os

In [2]:
# instantiate the client and model for search
search_model = AlephAlphaModel.from_model_name("luminous-base", token="API_TOKEN")

## Pre-defined functions
To make life a bit easier for you we have defined a few functions that you can use in this notebook.

They are described in this table:
|function|description|
|---|---|
|`embed_symmetric`| Embeds a text using the symmetric model|
|`embed_query`| Embeds a query using the asymmetric model|
|`embed_document`| Embeds a document using the asymmetric model|
|`cosine_similarity`| Calculates the cosine similarity between two vectors|

In [3]:
# function for symmetric embeddings 
def embed_symmetric(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Symmetric)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Queries
def embed_query(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Query)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Documents
def embed_document(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document)
    result = search_model.semantic_embed(request)
    return result.embedding

# function to calculate similarity
def cosine_similarity(v1: Sequence[float], v2: Sequence[float]) -> float:
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

In [4]:
# Texts to be used for searching
texts = [
    "The sun is shining. I was walking down the street when I saw an elefant in the park.",
    "An elefant is a mammal. It is a very large animal. Elefants are very intelligent.",
    "What is the meaning of 'elefant in a porcelin shop'?",
    ]

# The query that is supposed to be answered
query = "What is an elefant?"

First, let's create the symmetric and asymmetric embeddings for all the texts and the query.

If you are struggling with creating all the embeddings, try to use following loop:
```python
for text in texts:
    ...
```

In [5]:
# Task calculate the symmetric query embeddings for the query
symmetric_query = embed_symmetric(query)

# Task calculate the asymmetric query embeddings for the query
asymmetric_query = embed_query(query)

# Task generate symmetric embeddings for the texts
symmetric_embeddings = [embed_symmetric(text) for text in texts]

# Task generate asymmetric document embeddings for the texts
asymmetric_embeddings = [embed_document(text) for text in texts]

Now let's get the similarity between the query and the texts for both the symmetric and asymmetric embeddings.

The best way is to run them through a loop and print the score for each text.

In [6]:
# calculate the symmetric cosine similarity between the query and the texts
for i, text in enumerate(texts):
    print(f"Symmetric similarity between '{query}' and '{text[:25]}...': {cosine_similarity(symmetric_query, symmetric_embeddings[i])}")

Symmetric similarity between 'What is an elefant?' and 'The sun is shining. I was...': 0.41088780212108134
Symmetric similarity between 'What is an elefant?' and 'An elefant is a mammal. I...': 0.7456735307438987
Symmetric similarity between 'What is an elefant?' and 'What is the meaning of 'e...': 0.7191446960382698


In [22]:
# Task: calculate the asymmetric cosine similarity between the query and the texts
for i, text in enumerate(texts):
    print(f"Asymmetric similarity between '{query}' and '{text[:25]}...': {cosine_similarity(asymmetric_query, asymmetric_embeddings[i])}")

Asymmetric similarity between 'What is an elefant?' and 'The sun is shining. I was...': 0.46296463471994104
Asymmetric similarity between 'What is an elefant?' and 'An elefant is a mammal. I...': 0.7491473959096201
Asymmetric similarity between 'What is an elefant?' and 'What is the meaning of 'e...': 0.5931727226129333


## Working with longer texts

Now let's try to work with longer texts.

The length of texts is limited to 2048 tokens at a time, which translates to around 500 - 1000 words.

Therefore it often makes sense to split the text into smaller chunks and then calculate a combined similarity score.

In [12]:
query = "Who developed the first functional networks"

text_a = """Artificial neural networks (ANNs), usually simply called neural networks (NNs) or neural nets,[1] are computing systems inspired by the biological neural networks that constitute animal brains.[2]  
An ANN is based on a collection of connected units or nodes called artificial neurons, which loosely model the neurons in a biological brain. Each connection, like the synapses in a biological brain, can transmit a signal to other neurons. An artificial neuron receives signals then processes them and can signal neurons connected to it. The "signal" at a connection is a real number, and the output of each neuron is computed by some non-linear function of the sum of its inputs. 
The connections are called edges. Neurons and edges typically have a weight that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a connection. Neurons may have a threshold such that a signal is sent only if the aggregate signal crosses that threshold.  Typically, neurons are aggregated into layers. Different layers may perform different transformations on their inputs. Signals travel from the first layer (the input layer), to the last layer (the output layer), possibly after traversing the layers multiple times.
Training Neural networks learn (or are trained) by processing examples, each of which contains a known "input" and "result," forming probability-weighted associations between the two, which are stored within the data structure of the net itself. The training of a neural network from a given example is usually conducted by determining the difference between the processed output of the network (often a prediction) and a target output. This difference is the error. The network then adjusts its weighted associations according to a learning rule and using this error value. Successive adjustments will cause the neural network to produce output which is increasingly similar to the target output. After a sufficient number of these adjustments the training can be terminated based upon certain criteria. This is known as supervised learning.  
Such systems "learn" to perform tasks by considering examples, generally without being programmed with task-specific rules. For example, in image recognition, they might learn to identify images that contain cats by analyzing example images that have been manually labeled as "cat" or "no cat" and using the results to identify cats in other images. They do this without any prior knowledge of cats, for example, that they have fur, tails, whiskers, and cat-like faces. Instead, they automatically generate identifying characteristics from the examples that they process.  
History of artificial neural networks Warren McCulloch and Walter Pitts[3] (1943) opened the subject by creating a computational model for neural networks.[4] In the late 1940s, D. O. Hebb[5] created a learning hypothesis based on the mechanism of neural plasticity that became known as Hebbian learning. Farley and Wesley A. Clark[6] (1954) first used computational machines, then called "calculators", to simulate a Hebbian network. 
In 1958, psychologist Frank Rosenblatt invented the perceptron, the first artificial neural network,[7][8][9][10] funded by the United States Office of Naval Research.[11] The first functional networks with many layers were published by Ivakhnenko and Lapa in 1965, as the Group Method of Data Handling.[12][13][14] The basics of continuous backpropagation[12][15][16][17] were derived in the context of control theory by Kelley[18] in 1960 and by Bryson in 1961,[19] using principles of dynamic programming. Thereafter research stagnated following Minsky and Papert (1969),[20] who discovered that basic perceptrons were incapable of processing the exclusive-or circuit and that computers lacked sufficient power to process useful neural networks.  
In 1970, Seppo Linnainmaa published the general method for automatic differentiation (AD) of discrete connected networks of nested differentiable functions.[21][22] In 1973, Dreyfus used backpropagation to adapt parameters of controllers in proportion to error gradients.[23] Werbos\'s (1975) backpropagation algorithm enabled practical training of multi-layer networks. In 1982, he applied Linnainmaa\'s AD method to neural networks in the way that became widely used.[15][24]  The development of metal–oxide–semiconductor (MOS) very-large-scale integration (VLSI), in the form of complementary MOS (CMOS) technology, enabled increasing MOS transistor counts in digital electronics. This provided more processing power for the development of practical artificial neural networks in the 1980s.[25]  
In 1986 Rumelhart, Hinton and Williams showed that backpropagation learned interesting internal representations of words as feature vectors when trained to predict the next word in a sequence.[26]  From 1988 onward,[27][28] the use of neural networks transformed the field of protein structure prediction, in particular when the first cascading networks were trained on profiles (matrices) produced by multiple sequence alignments.[29]  In 1992, max-pooling was introduced to help with least-shift invariance and tolerance to deformation to aid 3D object recognition.[30][31][32] Schmidhuber adopted a multi-level hierarchy of networks (1992) pre-trained one level at a time by unsupervised learning and fine-tuned by backpropagation.[33]"""

In [21]:
# split text_a into splits and create an embedding for each split

text_splits = text_a.split("\n")

embeddings_text_a = [embed_document(text) for text in text_splits]

# embed the query
query_embedding = embed_query(query)

Now we can search for the split with the highest similarity score and not use the whole text.

In [26]:
# Search for the most similar split in text_a to the query and output its index
top_index = np.argmax([cosine_similarity(query_embedding, embedding) for embedding in embeddings_text_a])

print(f"The most similar split to the query is at index {top_index}:\n {text_splits[top_index]}")

The most similar split to the query is at index 6:
 In 1958, psychologist Frank Rosenblatt invented the perceptron, the first artificial neural network,[7][8][9][10] funded by the United States Office of Naval Research.[11] The first functional networks with many layers were published by Ivakhnenko and Lapa in 1965, as the Group Method of Data Handling.[12][13][14] The basics of continuous backpropagation[12][15][16][17] were derived in the context of control theory by Kelley[18] in 1960 and by Bryson in 1961,[19] using principles of dynamic programming. Thereafter research stagnated following Minsky and Papert (1969),[20] who discovered that basic perceptrons were incapable of processing the exclusive-or circuit and that computers lacked sufficient power to process useful neural networks.  
